In [1]:
# By Matt Stirling
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from ast import literal_eval

pd.set_option('display.width', 1400)

In [ ]:
# Load recipes into dataframe
converters = { k: literal_eval for k in ['tags', 'ingredients', 'steps', 'nutrition'] } # for evaluating strings as arrays (eg. tags)
df_recipes = pd.read_csv('dataset/RAW_recipes.csv', converters=converters)
df_recipes.set_index('id', inplace=True)
df_recipes['n_tags'] = df_recipes['tags'].apply( lambda tags: len(tags) ) # add n_tags column
df_recipes.head()

In [ ]:
# Load interactions (user ratings) into dataframe
df_interact = pd.read_csv('dataset/RAW_interactions.csv')
df_interact.head()

#### 3. Sentiment Analysis on User Reviews: Perform Sentiment Analysis on the user reviews using pre-trained models (e.g.,VADER, TextBlob, or BERT) to analyzes the sentiment polarity of each review and categorizes them into positive, neutral, or negative sentiments.

In [16]:
from textblob import TextBlob

In [22]:
def get_sentiment_textblob(text, buffer = 0):
    if isinstance(text, str):
        analysis = TextBlob(text)

        if analysis.sentiment.polarity > buffer:
            return 'positive'
        elif analysis.sentiment.polarity < -buffer:
            return 'negative'
        
    return 'neutral'

In [36]:
reviews_to_analyze = 15
reviews_to_show = 15

df_interact_subset = df_interact.head(reviews_to_analyze).copy()
df_interact_subset.loc[:, 'sentiment'] = df_interact_subset['review'].apply(get_sentiment_textblob)
df_interact_subset.head(reviews_to_show)

,user_id,recipe_id,date,rating,review,sentiment
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...,positive
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall...",positive
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...,positive
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...,positive
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin...",negative
...,...,...,...,...,...,...
145,44642,190875,2006-10-19,0,"Just posting, so I will know I tryed this. Nev...",neutral
146,162191,190875,2006-10-20,5,I loved this recipe.,positive
147,137302,224025,2012-04-18,2,This is probably one of those toppings which h...,positive
148,238496,254859,2007-10-04,4,Delicious and fresh are the words I would choo...,positive
